# 1. 利用梯度磁带计算导数

In [1]:
import tensorflow as tf
import numpy as np

# 计算函数 f(x) = a*x**2 + b*x + c 在x=2.0处的导数

x = tf.Variable(2.0, name='x', dtype=tf.float32)
a = tf.constant(1.0)
b = tf.constant(-2.0)
c = tf.constant(1.0)

with tf.GradientTape() as tape:
    y = a*tf.pow(x, 2) + b*x + c
    
dy_dx = tape.gradient(y, x)
print(dy_dx)

tf.Tensor(2.0, shape=(), dtype=float32)


In [10]:
#　对常量张量求导，需要增加watch
with tf.GradientTape() as tape:
    tape.watch([a, b, c])
    y = a*tf.pow(x, 2) + b*x + c

dy_dx, dy_da, dy_db, dy_dc = tape.gradient(y, [x, a, b, c])
print(dy_dx.numpy())
print(dy_da)
print(dy_db)
print(dy_dc)

2.0
tf.Tensor(4.0, shape=(), dtype=float32)
tf.Tensor(2.0, shape=(), dtype=float32)
tf.Tensor(1.0, shape=(), dtype=float32)


In [11]:
# 求二阶导数
with tf.GradientTape() as tape2:
    with tf.GradientTape() as tape1:
        y = a*tf.pow(x, 2) + b*x + c
    dy_dx = tape1.gradient(y, x)
dy2_dx2 = tape2.gradient(dy_dx, x)

print(dy2_dx2)

tf.Tensor(2.0, shape=(), dtype=float32)


In [16]:
# 在autograph中使用
@tf.function
def f(x):
    a = tf.constant(1.0)
    b = tf.constant(-2.0)
    c = tf.constant(1.0)
    
    # 利用cast将x转为float32
    x = tf.cast(x, tf.float32)
    with tf.GradientTape() as tape:
        tape.watch(x)
        y = a*tf.pow(x, 2) + b*x +c
    dy_dx = tape.gradient(y, x)
    
    return (dy_dx, y)

print(f(tf.constant(1)))
print(f(tf.constant(2)))

(<tf.Tensor: id=304, shape=(), dtype=float32, numpy=0.0>, <tf.Tensor: id=305, shape=(), dtype=float32, numpy=0.0>)
(<tf.Tensor: id=307, shape=(), dtype=float32, numpy=2.0>, <tf.Tensor: id=308, shape=(), dtype=float32, numpy=1.0>)


# 2. 利用梯度磁带和优化器求最小值

In [20]:
# 使用optimizer.apply_gradients求函数f(x) = a*x**2 + b*x +c 的最小值
x = tf.Variable(0.0, name='x', dtype=tf.float32)
a = tf.constant(1.0)
b = tf.constant(-2.0)
c = tf.constant(1.0)

optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
for _ in range(1000):
    with tf.GradientTape() as tape:
        y = a*tf.pow(x, 2) + b*x + c
    dy_dx = tape.gradient(y, x)
    optimizer.apply_gradients(grads_and_vars=[(dy_dx, x)])

tf.print('y =', y, ', x =', x)

y = 0 , x = 0.999998569


In [22]:
# 使用optimizer.minimize求函数最小值
# optimizer.minimize相当于先用tape求gradient，再apply_gradients
x = tf.Variable(0.0, name='x', dtype=tf.float32)

def f():
    a = tf.constant(1.0)
    b = tf.constant(-2.0)
    c = tf.constant(1.0)
    y = a*tf.pow(x, 2) + b*x + c
    return y

optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
for _ in range(1000):
    optimizer.minimize(f, [x])

tf.print('y =', y, ', x =', x)

y = 0 , x = 0.999998569


In [29]:
# 在autograph中使用optimizer.apply_gradients求函数f(x) = a*x**2 + b*x +c 的最小值
x = tf.Variable(0.0, name='x', dtype=tf.float32)
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)

@tf.function
def minimizef():
    a = tf.constant(1.0)
    b = tf.constant(-2.0)
    c = tf.constant(1.0)
    
    for _ in tf.range(1000):
        with tf.GradientTape() as tape:
            y = a*tf.pow(x, 2) + b*x + c
        dy_dx = tape.gradient(y, x)
        optimizer.apply_gradients(grads_and_vars=[(dy_dx, x)])
    y = a*tf.pow(x, 2) + b*x + c
    return y
        
tf.print(minimizef())
tf.print(x)

0
0.999998569


In [32]:
# 在autograph中使用optimizer.minimize求函数f(x) = a*x**2 + b*x +c 的最小值
x = tf.Variable(0.0, name='x', dtype=tf.float32)
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)

@tf.function
def f():
    a = tf.constant(1.0)
    b = tf.constant(-2.0)
    c = tf.constant(1.0)
    y = a*tf.pow(x, 2) + b*x + c
    return y

@tf.function
def train():
    for _ in range(1000):
        optimizer.minimize(f, [x])
    return f()

tf.print(train())
tf.print(x)

0
0.999998569
